In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
df = pd.read_csv('X_train.csv')

# UserName

In [3]:
t = {}
i = 1

for user in df['userName'].unique():
    t[user] = i
    i += 1
    
for user in t:
    df.set_value(df['userName'] == user, 'userName', t[user])
    
df['userName'] = pd.to_numeric(df['userName'])

# Comments

In [4]:
import itertools
import operator
from collections import Counter

In [5]:
bad = df[df['reting'] <= 3.5]['comment']
splits = list(map(lambda x: x.split(' '), bad))
arr = list(itertools.chain.from_iterable(splits))
bad = dict(Counter(arr))
sorted_bad = sorted(bad.items(), key=operator.itemgetter(1), reverse=True)
sorted_bad[: 20]

[('не', 4892),
 ('и', 4425),
 ('', 3695),
 ('в', 3279),
 ('на', 2758),
 ('-', 1722),
 ('что', 1616),
 ('с', 1433),
 ('но', 1050),
 ('а', 918),
 ('как', 808),
 ('очень', 798),
 ('по', 767),
 ('это', 766),
 ('для', 765),
 ('при', 721),
 ('только', 638),
 ('через', 623),
 ('все', 607),
 ('так', 584)]

In [6]:
good = df[df['reting'] > 3.5]['comment']
splits = list(map(lambda x: x.split(' '), good))
arr = list(itertools.chain.from_iterable(splits))
good = dict(Counter(arr))
sorted_good = sorted(good.items(), key=operator.itemgetter(1), reverse=True)
sorted_good[: 20]

[('и', 16197),
 ('не', 13812),
 ('', 13092),
 ('в', 11128),
 ('на', 8793),
 ('-', 5509),
 ('с', 5258),
 ('что', 5182),
 ('очень', 4425),
 ('но', 3401),
 ('для', 3360),
 ('как', 2720),
 ('все', 2676),
 ('по', 2567),
 ('это', 2524),
 ('а', 2331),
 ('у', 2173),
 ('за', 2157),
 ('я', 1916),
 ('так', 1771)]

# Properties

In [173]:
df = pd.read_csv('X_train.csv')

In [174]:
for i in range(0, df.shape[0]):
    props = df['property'][i].split(', ')
    q = map(lambda x: x.split(': '), props)
    props_new = list(q)
    
    res = []

    for j in range(0, len(props_new)):
        res.append(props_new[j][0].split('{')[1])
    
    df.set_value(i, 'property', res)

## Select properties

In [175]:
good = df[df['reting'] > 3.5]['property']
arr = list(itertools.chain.from_iterable(good))
good = dict(Counter(arr))

props = pd.DataFrame()
props['props'] = good.keys()
props['count'] = good.values()

props_good = list(props[(props['count'] < 500) & (props['count'] > 300)]['props'].astype(int))
#props_good = list(props[(props['count'] > 1000)]['props'].astype(int))
len(props_good)

79

In [176]:
bad = df[df['reting'] <= 3.5]['property']
arr = list(itertools.chain.from_iterable(bad))
bad = dict(Counter(arr))

props = pd.DataFrame()
props['props'] = bad.keys()
props['count'] = bad.values()

props_bad = list(props[(props['count'] < 900) & (props['count'] > 200)]['props'].astype(int))
#props_bad = list(props[(props['count'] > 1000)]['props'].astype(int))
len(props_bad)

55

In [177]:
props_good_new = set(props_good) - set(props_bad)
props_bad_new = set(props_bad) - set(props_good)

props_good = props_good_new
props_bad = props_bad_new

print(len(props_good))
print(len(props_bad))

77
53


In [178]:
for j in range(0, df.shape[0]):
    foo = df['property'][j]

    count_good = 0
    count_bad = 0

    for i in range(0, len(foo)):
        if int(foo[i]) in props_good:
            count_good += 1

        if int(foo[i]) in props_bad:
            count_bad += 1
    
#     if count_good > 3:
#         df.set_value(j, 'is_good_probs', count_good)#True)
#     else:
#         df.set_value(j, 'is_good_probs', False)
        
#     if count_bad > 3:
#         df.set_value(j, 'is_bad_probs', count_bad)#True)
#     else:
#         df.set_value(j, 'is_good_probs', False)
        
    df.set_value(j, 'is_good_probs', int(count_good))
    df.set_value(j, 'is_bad_probs', int(count_bad))

In [179]:
df_prop = df[['reting', 'is_good_probs', 'is_bad_probs']].astype(int)
#df_prop

In [180]:
limit = 10

In [181]:
# много свойств в плохом и плохой рейтинг

df[(df['is_bad_probs'] > limit) & (df['reting'] < 4)].shape[0]

399

In [182]:
# много свойств в плохом и хороший рейтинг

df[(df['is_bad_probs'] > limit) & (df['reting'] > 3)].shape[0]

1561

In [183]:
# много свойств в хорошем и хороший рейтинг

df[(df['is_good_probs'] > limit) & (df['reting'] > 3)].shape[0]

705

In [184]:
# много свойств в хорошем и плохой рейтинг

df[(df['is_good_probs'] > limit) & (df['reting'] < 4)].shape[0]

205

In [185]:
for i in range(0, df.shape[0]):
    if df['is_bad_probs'][i] > limit:
        df['is_good_bool'] = True
    else:
        df['is_good_bool'] = False

# Prediction

In [19]:
y = df['reting'].astype(int)
X = df.drop(['categoryLevel1Id', 'reting', 'date', 'commentNegative', 'commentPositive'], axis=1)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [17]:
boost = xgb.XGBClassifier(max_depth=15, n_estimators=200)
boost.fit(X_train, y_train)
y_pred = boost.predict(X_test)
print(classification_report(list(y_test), list(y_pred)))

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in fields property, comment

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
tree = ExtraTreeClassifier()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
est = xgb.XGBRegressor()
est.fit(X_train, y_train)
y_pred = est.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
X